#### Assignment 4 - Question 4: Homography

#### Setting up

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
img1 = cv2.imread('hallway1.jpg')
img2 = cv2.imread('hallway2.jpg')
img3 = cv2.imread('hallway3.jpg')

img1 = cv2.cvtColor(img1, cv2.COLOR_RGB2BGR)
img2 = cv2.cvtColor(img2, cv2.COLOR_RGB2BGR)
img3 = cv2.cvtColor(img3, cv2.COLOR_RGB2BGR)

#### Step 1: Plot Selected Corresponding Points

In [ ]:
def get_correspondence(case):
    if case == "A":
        pts1 = np.array([[999, 76],[1062, 13],[1095,228],[790, 614],[838,660]])
        pts2 = np.array([[845, 391],[900, 331],[946, 537],[666, 937],[717, 981]])
    elif case == "B":
        pts1 = np.array([[999, 75],[1061, 12],[948,392],[788, 614],[1162,793]])
        pts2 = np.array([[902, 266],[936,204],[882,582],[795,807],[1022,986]])
    elif case == "C":
        pts1 = np.array([[653,544],[789,614],[840,660],[483,777],[536,645]])
        pts2 = np.array([[687,740],[795,807],[826,854],[415,982],[528,845]])
    else:
        return 
    return pts1, pts2



In [ ]:
def plot_correspondence(case, image1, image2):
    pts1, pts2 = get_correspondence(case)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 10))
    fig.suptitle(f'Case {case} Correspondences', y=0.7, fontsize=12)

    gray1 = cv2.cvtColor(image1, cv2.COLOR_RGB2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_RGB2GRAY)

    ax1.imshow(gray1, cmap='gray')
    ax2.imshow(gray2, cmap='gray')

    for i in range(5):
        ax1.plot(pts1[i, 0], pts1[i, 1], "rs", fillstyle="none")
        ax2.plot(pts2[i, 0], pts2[i, 1], "rs", fillstyle="none")
    plt.show()


In [ ]:
plot_correspondence("A", img1, img2)

In [ ]:
plot_correspondence("B", img1, img3)

In [ ]:
plot_correspondence("C", img1, img3)

#### Step 2: Fit a Homography

In [ ]:
def fit_homography(pts1, pts2):
    n = len(pts1)
    A = np.zeros((2*n, 9))
    for i in range(5):
        x1, y1 = pts1[i, 0], pts1[i, 1]
        x2, y2 = pts2[i, 0], pts2[i, 1]
        A[2*i] = [-x1, -y1, -1, 0, 0, 0, x1*x2, y1*x2, x2]
        A[2*i+1] = [0, 0, 0, -x1, -y1, -1, x1*y2, y1*y2, y2]

    # Solve for the homography matrix
    U, S, Vt = np.linalg.svd(A)
    L = Vt[-1] / Vt[-1, -1]
    H = L.reshape(3, 3)

    return H

def find_case_homo(case):
    pts1, pts2 = get_correspondence(case)
    return fit_homography(pts1, pts2)


##### Case A

In [ ]:
find_case_homo("A")

Comment:
- Translation: the translation vector is approximately (-238.57, 311.97), indicating that the transformed image has been shifted to the left and up relative to the original image.
- Scale: the scaling factors are 1.18 and 1.055 respectively for x and y directions, meaning the image is slightly up-scaled.
- Shear: very small figures that can be ignored
- Rotation: very small figures.

##### Case B

In [ ]:
find_case_homo("B")

Comment:
- Translation: the translation vector is approximately (311.95, 192.81), indicating that the transformed image has been shifted to right and up.
- Scale: the scaling factors are 0.598 and 0.991 respectively for x and y directions, meaning the image is down-sampled.
- Shear: shearing factors (0.0123, 0.0013) means the figure is slightly sheared in both directions.
- Rotation: very small figures that can be ignored 

##### Case C

In [ ]:
find_case_homo("C")

Comment:
- Translation: the translation vector is approximately (333.14, 285.35), indicating that the transformed image has been shifted to the right and up relative to the original image.
- Scale: the image has been downsampled by 0.787 in each direction.  
- Rotation: transformed image has been rotated counterclockwise
- Shear: -0.4585 and -0.1072 represent shear to the left and down

#### Step 3: Use Homography to Map Selected Point to Estimated Point

In [ ]:
def get_mapped_coordinate(H, x, y):

    v = np.array([x, y, 1.])
    v = H @ v
    z = v[2]
    v = v / z
    est_x, est_y = round(v[0]), round(v[1])

    return est_x, est_y

def estimate_coordinates(pts, H):
    
    estimated = []

    for pt in pts:
        x, y = pt
        est_x, est_y = get_mapped_coordinate(H, x, y)
        estimated.append([est_x, est_y])

    return np.array(estimated)

In [ ]:
def plot_est_homo(case, image):
    img_copy = image.copy()

    pts1, pts2 = get_correspondence(case)
    H = fit_homography(pts1, pts2)
    pts_est = estimate_coordinates(pts1, H)
    
    fig, ax = plt.subplots()

    ax.imshow(img_copy)

    for i in range(5):
        ax.plot(pts_est[i, 0], pts_est[i, 1], "gs", fillstyle="none", markersize=10)
    for i in range(5):  
        ax.plot(pts2[i, 0], pts2[i, 1], "rs", fillstyle="none", markersize=13)
    
    plt.show()


def plot_est_homo_case(case):
    if case == "A":
        plot_est_homo("A", img2)
    elif case == "B":
        plot_est_homo("B", img3)
    elif case == "C":
        plot_est_homo("C", img3)
    else:
        return

In [ ]:
plot_est_homo_case("A")

In [ ]:
plot_est_homo_case("B")


In [ ]:
plot_est_homo_case("C")


#### Step 4: Inverse Mapped Pixels

In [ ]:
def inverse_mapped_pixel(case, image1, image2, plot=True):
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    pts1, pts2 = get_correspondence(case)
    H = fit_homography(pts1, pts2)
    pts_est = estimate_coordinates(pts1, H)

    # initialize to zero
    padding = 500
    x, y, z = gray1.shape[0] + padding * 2, gray1.shape[1] + padding * 2,  3

    # fill red channel with values of the original image
    new_img = np.zeros((x, y, z))
    new_img[padding:x-padding, padding:y-padding, 0] = gray1

    # blue and green channels

    for x in range(new_img.shape[0]):
        for y in range(new_img.shape[1]):
            y_, x_ = get_mapped_coordinate(H, y, x) # note x, y is reversed
            if x_ >= padding and x_ < padding + gray2.shape[0] \
                and y_ >= padding and y_ < padding + gray2.shape[1]:
                new_img[x, y, 1] = gray2[x_ - padding, y_ - padding]
                new_img[x, y, 2] = gray2[x_ - padding, y_ - padding]
    new_img /= 255
    if plot:
        plt.imshow(new_img)
    else:
        return new_img
    

In [ ]:
inverse_mapped_pixel("A", img1, img2)

In [ ]:
inverse_mapped_pixel("B", img1, img3)

In [ ]:
inverse_mapped_pixel("C", img1, img3)

- Homography contains perspective transformations consist of scale, translation, shear, and rotation
- Camera position is about the same in image1 and image2, with camera orientation up in image2.
- Camera positions are different in image1 and image3, also camera orientation is slightly counter-clockwisely rotated.
- The floor is of less Lambertian reflectance while the right wall has more Lambertian reflectance. As we can observe from the three images, the floor has a "mirror-like" reflections while the right wall is more of matte texture.

#### Step 5: Putting together

In [ ]:
case = input("What's the case you want to run? Type 'A', 'B', or 'C' without quotation marks")

In [ ]:
def run(case, image1, image2):
    fig, axes = plt.subplots(2, 2,figsize=(15,15))
    fig.suptitle(f'Case {case}',y=0.85, fontsize=16)

    pts1, pts2 = get_correspondence(case)

    gray1 = cv2.cvtColor(image1, cv2.COLOR_RGB2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_RGB2GRAY)

    axes[0,0].imshow(image1)
    axes[0,0].set_title('4.1: 1st Image Selected Points')
    axes[0,1].imshow(image2)
    axes[0,1].set_title('4.1: 2nd Image Selected Points')

    for i in range(5):
        axes[0,0].plot(pts1[i, 0], pts1[i, 1], "rs", fillstyle="none")
        axes[0,1].plot(pts2[i, 0], pts2[i, 1], "rs", fillstyle="none")

    H = fit_homography(pts1, pts2)

    pts_est = estimate_coordinates(pts1, H)

    axes[1,0].imshow(image2)
    axes[1,0].set_title('4.3: Estimated Points Using Homography')
    for i in range(5):
        axes[1,0].plot(pts_est[i, 0], pts_est[i, 1], "gs", fillstyle="none", markersize=10)
    for i in range(5):  
        axes[1,0].plot(pts2[i, 0], pts2[i, 1], "rs", fillstyle="none", markersize=13)
    
    new_img = inverse_mapped_pixel(case, image1, image2, False)
    axes[1,1].imshow(new_img)
    axes[1,1].set_title('4.4: Inverse Mapped Pixels')

    plt.show()

    

In [ ]:
if case == "A":    
    run("A", img1, img2)
elif case == "B":
    run("B", img1, img3)
elif case == "C":
     run("C", img1, img3)
else:
     print("Invalid case number")